<a href="https://colab.research.google.com/github/Andre-1970/Machine_learning_classical_algorithms_Sem4/blob/main/3_11_%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D1%8F%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Домашняя работа

**Задание простого уровня** Мы говорили, что метрики качества нужны, чтобы сравнивать различные модели между собой. В задаче полиномиальной регрессии реализуйте код для выбора лучшей степени полиному:

* возьмите все степени от 1 до 10 по порядку, без пропусков.
* найдите степень полинома, где будет лучший r2-score
* напишите код, который выводит самую подходящую степень полинома и соответствующий ей скор

Эта процедура называется Grid Search и помогает найти лучшие параметры для модели.

Обучите лучшую модель и сделайте predict

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore

In [6]:
degrees = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
best_score = 0.0
best_degree = 0

In [9]:
df = pd.read_csv('data/phones.csv')
# разделение признаков и целевой переменной
X = df[['disk', 'year']]
y = df['price']

In [10]:
# Переберем все степени и найдем лучший R2-score
for degree in degrees:
    # Создадим полиномиальные признаки
    poly_features = PolynomialFeatures(degree)
    X_poly = poly_features.fit_transform(X)
    # Обучим модель
    model = LinearRegression()
    model.fit(X_poly, y)

    # Вычислим R2-score
    score = r2_score(y, model.predict(X_poly))

    # Если R2-score лучше, чем текущий лучший, обновим переменные
    if score > best_score:
        best_score = score
        best_degree = degree

In [11]:
# Выведем лучшую степень и соответствующий ей R2-score
print("Best degree:", best_degree)
print("Best R2-score:", best_score)

# Обучим лучшую модель и построим предсказания
poly_features = PolynomialFeatures(best_degree)
X_poly = poly_features.fit_transform(X)
model = LinearRegression()
model.fit(X_poly, y)
y_pred = model.predict(X_poly)

# Выведем предсказания
print("Predictions:", y_pred)

Best degree: 9
Best R2-score: 0.9943138168888681
Predictions: [ 7562.57611915  7345.33051655  1677.40295002 14835.58109257
  4043.55809948 15810.70185539  7345.33051655  4658.04765007
 12707.54276535  6582.52661106 13790.47771332 10759.59111091
  9557.69453308  5214.90372536  7008.15501282  2250.62603542
 12707.54276535  5998.16981384 13158.91097996 12090.86844704
  1383.44454643  4658.04765007 10558.26303837  9620.38674137
  8571.01982281  6582.52661106  8441.97591278  9038.25140259
  2380.7844331  12707.54276535 11619.22429058  4870.86700889
  7008.15501282  4994.32917377  7562.57611915  5958.78547356
 12707.54276535 15810.70185539  7562.57611915  4658.04765007
 14835.58109257 12707.54276535  5257.93419525  8441.97591278
  7562.57611915  7345.33051655  9620.38674137  2699.55728886
 10558.26303837 13158.91097996 10042.74772903  8030.69995758
  3133.17899963  1677.40295002  8030.69995758  7871.35931179
  1431.85556766 12707.54276535 10558.26303837 12090.86844704
 10558.26303837  4658.0

**Задание среднего уровня** Напишите класс для обучения модели, который содержит:

* функцию `.fit(X, y)` , которая принимает на вход массив фичей `X`, массив таргетов `y` и обучает коэффициенты регрессии. Код для обучения взять из первого урока модуля *Постановка ML задачи линейной регрессии*
* функцию `.predict(X)`, которая по массиву фичей `X` возвращает массив предсказаний `y`

Нужно использовать код для аналитически вычисляемых коэффициентов. 

Это задание позволит понять, как работает линейная регрессия "внутри" библиотечной реализации.

In [22]:
class CustomLinearReg:
    def __init__(self):
        self.coef_ = None
        self.intercept_ = None

    def fit(self, X, y):
        X = np.hstack((np.ones((X.shape[0], 1)), X))
        self.coef_ = np.linalg.inv(X.T @ X) @ X.T @ y
        self.intercept_ = self.coef_[0]
        self.coef_ = self.coef_[1:]

    def predict(self, X):
        X = np.hstack((np.ones((X.shape[0], 1)), X))
        return X @ np.hstack((self.intercept_, self.coef_))

In [23]:
df = pd.read_csv('data/phones.csv')
# разделение признаков и целевой переменной
X = df[['disk', 'year']]
y = df['price']

# разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = CustomLinearReg()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_test, y_pred))}')

MAE: 370.5450395136257
MSE: 207228.6017281483
RMSE: 455.2236831802013


**Задание высокого уровня**

1. разделите датасет с домами Бостона из Урока 2 (таргет и фичи) на две части: в одной части 80% датасета (назовём train) в другой 20% (назовём valid) с помощью функции `train_test_split` из библиотеки `sklearn`
1. обучите модель только на train датасете
1. постройте предсказания valid датасете
1. Посчитайте  `r2 score` на валидационном сете

После этого примените к обеим датасетам z-преобразование и повторите шаги 2-4. Как изменилась метрика r2?

Это задание поможет понять, как валидировать линейную регрессию (и другие модели) на отложенной выборке.

In [19]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('data/phones.csv')
# разделение признаков и целевой переменной
X = df[['disk', 'year']]
y = df['price']

# делим датасет на train и valid
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
#1
model = LinearRegression()
model.fit(X_train, y_train)

#2
y_pred = model.predict(X_valid)

#3
score_before = r2_score(y_valid, y_pred)

#4
print(f"R2 до преобразования: {score_before:.4f}")

R2 до преобразования: 0.9788


In [21]:
X_train_z = X_train.apply(zscore)
X_valid_z = X_valid.apply(zscore)

#1
model.fit(X_train_z, y_train)

#2
y_pred_valid_z = model.predict(X_valid_z)

#3
score_after = r2_score(y_valid, y_pred_valid_z)

#4
print(f"R2 после преобразования: {score_before:.4f}")

R2 после преобразования: 0.9788


In [24]:
print(f'Изменение метрики R2: {score_before-score_after}')

Изменение метрики R2: 0.0009132858981966141
